In [1]:
import pandas as pd
from tsfresh import extract_relevant_features, extract_features
import numpy as np

/opt/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
dataset = pd.read_csv('../data/PeMS/Incidents/work_folder/windows.csv')

In [26]:
sample = dataset.loc[:10].copy()

In [33]:
def feature_extraction(dataset):
    cols = [col for col in dataset.columns if 'Avg' in col or 'Total' in col]
    dataset = dataset[cols]
    t_labels = np.arange(len(cols)/3, dtype=int)
    t_labels = np.concatenate((t_labels,t_labels,t_labels))
    aux_labels = pd.DataFrame({'val_name': cols, 't': t_labels})
    long = dataset.reset_index().melt(id_vars=['index'], var_name='series_param', value_name='value')
    long = long.merge(aux_labels, how='left', left_on='series_param', right_on='val_name').drop('val_name', axis=1)
    long['series_param'] = long['series_param'].apply(lambda row: row.split('_')[0])
    long = long.rename(columns={'index': 'ID'})[['ID', 't', 'series_param', 'value']]
    extracted = extract_features(long, column_id='ID', column_sort='t', column_kind='series_param', column_value='value', n_jobs=4)
    # try imputation
    extracted.dropna(axis=1, how='any', inplace=True)
    return extracted

In [49]:
def batch_extraction(dataset, b_size):
    m=dataset.shape[0]
    result=pd.DataFrame()
    for i in range(0, m, b_size):
        crnt_ftrs = feature_extraction(dataset.loc[i:i+b_size-1])
        result = pd.concat((result, crnt_ftrs))
    print('EGOR virubajjj!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    return result

In [50]:
extracted = batch_extraction(dataset.loc[:3500], 1000)

Feature Extraction: 100%|██████████| 20/20 [00:46<00:00,  2.35s/it]


EGOR virubajjj!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [54]:
extracted.tail()

,Avg Occupancy__abs_energy,Avg Occupancy__absolute_sum_of_changes,"Avg Occupancy__agg_autocorrelation__f_agg_""mean""","Avg Occupancy__agg_autocorrelation__f_agg_""median""","Avg Occupancy__agg_autocorrelation__f_agg_""var""","Avg Occupancy__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","Avg Occupancy__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","Avg Occupancy__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","Avg Occupancy__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""stderr""","Avg Occupancy__agg_linear_trend__f_agg_""max""__chunk_len_5__attr_""intercept""",...,Total Flow__time_reversal_asymmetry_statistic__lag_1,Total Flow__time_reversal_asymmetry_statistic__lag_2,Total Flow__time_reversal_asymmetry_statistic__lag_3,Total Flow__value_count__value_-inf,Total Flow__value_count__value_0,Total Flow__value_count__value_1,Total Flow__value_count__value_inf,Total Flow__value_count__value_nan,Total Flow__variance,Total Flow__variance_larger_than_standard_deviation
id,,,,,,,,,,,,,,,,,,,,,
3496,0.092094,0.02280,-0.252108,-0.019108,1.002403,0.07926,-1.0,-0.00088,0.0,0.078116,...,24461.613957,42639.217781,48506.934938,0.0,0.0,0.0,0.0,0.0,10.130100,1.0
3497,0.170914,0.05278,-0.057633,-0.003566,0.295902,0.11860,-1.0,-0.00726,0.0,0.106420,...,28153.192085,64555.134074,88400.384385,0.0,0.0,0.0,0.0,0.0,25.351649,1.0
3498,0.841568,0.52858,-0.261113,-0.325108,0.291994,0.24848,1.0,0.11834,0.0,0.221158,...,11497.751518,1846.665708,-23472.308025,0.0,0.0,0.0,0.0,0.0,32.397959,1.0
3499,1.241213,0.10892,-0.554886,-0.715460,0.843508,0.30160,1.0,0.02048,0.0,0.250144,...,-70789.364882,-146145.003451,-243859.506458,0.0,0.0,0.0,0.0,0.0,54.588494,1.0
3500,0.012169,0.01346,-0.399821,-0.755578,0.480029,0.03226,-1.0,-0.00698,0.0,0.031902,...,-4951.885714,-11108.642667,-18321.839200,0.0,0.0,0.0,0.0,0.0,20.282344,1.0


In [20]:
dataset.reset_index(drop=True, inplace=True)

In [30]:
m=dataset.shape[0]
step=1000
for i in range(0, m, step):
#     dataset.loc[i:i+step]
    print(i, i+step)

0 1000
1000 2000
2000 3000
3000 4000
4000 5000
5000 6000
6000 7000
7000 8000
8000 9000
9000 10000
10000 11000
11000 12000
12000 13000
13000 14000
14000 15000
15000 16000
16000 17000
17000 18000
18000 19000
19000 20000
20000 21000
21000 22000
22000 23000
23000 24000
24000 25000
25000 26000
26000 27000
27000 28000
28000 29000
29000 30000
30000 31000
31000 32000


In [8]:
extracted_short = feature_extraction(sample.iloc[:2])

Feature Extraction: 100%|██████████| 6/6 [00:00<00:00, 22.04it/s]


In [10]:
extracted_short.loc[0]

variable
Avg Occupancy__abs_energy                                                            0.595636
Avg Occupancy__absolute_sum_of_changes                                               0.053860
Avg Occupancy__agg_autocorrelation__f_agg_"mean"                                    -0.158590
Avg Occupancy__agg_autocorrelation__f_agg_"median"                                  -0.006676
Avg Occupancy__agg_autocorrelation__f_agg_"var"                                      0.613333
Avg Occupancy__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"intercept"         0.199040
Avg Occupancy__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"rvalue"            1.000000
Avg Occupancy__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"slope"             0.002260
Avg Occupancy__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"stderr"            0.000000
Avg Occupancy__agg_linear_trend__f_agg_"max"__chunk_len_5__attr_"intercept"          0.202770
Avg Occupancy__agg_linear_trend__f_agg_"max"__chunk

In [6]:
extracted.loc[0]

variable
Avg Occupancy__abs_energy                                                            0.595636
Avg Occupancy__absolute_sum_of_changes                                               0.053860
Avg Occupancy__agg_autocorrelation__f_agg_"mean"                                    -0.158590
Avg Occupancy__agg_autocorrelation__f_agg_"median"                                  -0.006676
Avg Occupancy__agg_autocorrelation__f_agg_"var"                                      0.613333
Avg Occupancy__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"intercept"         0.199040
Avg Occupancy__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"rvalue"            1.000000
Avg Occupancy__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"slope"             0.002260
Avg Occupancy__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"stderr"            0.000000
Avg Occupancy__agg_linear_trend__f_agg_"max"__chunk_len_5__attr_"intercept"          0.202770
Avg Occupancy__agg_linear_trend__f_agg_"max"__chunk

In [18]:
extracted.shape

(1001, 486)

In [12]:
extracted_short.shape

(2, 1228)